In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

In [2]:
def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        result=[]
        for word in words:
            word_l=word.lower()
            if word_l not in stopwords:
                result.append(word_l)
        return result
    except ValueError as e:
        return []

def bigrams(words):
    result=[]
    for w1,w2 in zip(words,words[1:]):
        result.append((w1+"_"+w2,1))
    return result

#wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16)\
#.map(parse_article)\
#.flatMap(bigrams)\
#.reduceByKey(lambda x,y:x+y)\
#.filter(lambda x: x[1]>=500)


In [3]:
stopwords = sc.textFile("/datasets/stop_words_en.txt", 16).collect()
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16)\
.map(parse_article).cache()

In [25]:
w_bigrams=wiki\
.flatMap(bigrams)\
.reduceByKey(lambda x,y:x+y)\
.cache()

In [ ]:
total_bigrams=w_bigrams.map(lambda x:x[1]).sum()

In [7]:
w_unigrams=wiki\
.flatMap(lambda x: x)\
.map(lambda x: (x,1))\
.reduceByKey(lambda x,y:x+y)\
.sortByKey()\
.cache()
#.sortBy(lambda x: x[0])
#list_unigrams=w_unigrams.collect()

In [8]:
total_unigrams=w_unigrams.map(lambda x:x[1]).sum()

In [9]:
broadcastVar = sc.broadcast(w_unigrams.collectAsMap())

In [29]:
from math import log
def npmi(input_bigram):
    bigram, count_ab=input_bigram
    a,b=bigram.split('_')
    count_a=broadcastVar.value[a]
    count_b=broadcastVar.value[b]
    
    pa=float(count_a)/total_unigrams
    pb=float(count_b)/total_unigrams
    pab=float(count_ab)/total_bigrams
    #PMI(a, b) = ln (P(ab) / (P(a) * P(b))
    pmi_ab=log(pab/(pa*pb))
    #NPMI(a, b) = PMI(a, b) / -ln P(ab)
    res_npmi=pmi_ab/(-1*log(pab))
    
    return (bigram,res_npmi)

result=w_bigrams.filter(lambda x: x[1]>500).map(npmi).sortBy(lambda x: x[1],ascending=False).cache()

In [31]:
for res in result.take(39):
    print(res[0])

los_angeles
external_links
united_states
prime_minister
san_francisco
et_al
new_york
supreme_court
19th_century
20th_century
references_external
soviet_union
air_force
baseball_player
university_press
roman_catholic
united_kingdom
references_reading
notes_references
award_best
north_america
new_zealand
civil_war
catholic_church
world_war
war_ii
south_africa
took_place
roman_empire
united_nations
american_singer-songwriter
high_school
american_actor
american_actress
american_baseball
york_city
american_football
years_later
north_american
